In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
from ipywidgets import *
import numpy as np
import k3d
import matplotlib.pyplot as plt
from sloppy.optic import *
from sloppy.raytracing import *
from sloppy.abcd import *
from sloppy.utils import *
from sloppy.tools import *

In [3]:
from cavities import *

In [5]:
cavity_parameter_interaction_factory(LensCavQuartic, 'lens_dist', 1e-1, N=100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatLogSlider(value=0.1, description='scanrange', layout=Layout(height='30px', width='8…

In [4]:
waists_vs_param(LensCavQuartic, 'lens_dist', 1e-1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

25.125183946488296

In [5]:
res = degeneracy_length(LensCavQuartic, 'lens_dist', 1e-1)
Ldeg = res.x
print(res)

     fun: 189015.5869453489
 message: 'Solution found.'
    nfev: 9
  status: 0
 success: True
       x: 25.12982681286276


In [6]:
elements = LensCavQuartic(lens_dist=Ldeg)
sys = RaySystem(elements)

In [7]:
system = sys.abcd
print(system.Ltot)
system.waist_at(0)

125.96287424209984


array([0.01982769, 0.01982769])

In [10]:
x = np.linspace(0, system.Ltot, 500)
ws = system.compute_waists(x)

plt.figure()
plt.plot(x, ws)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
mu = np.array([0., 0., 0., 0.])
ray = sys.screen.eigenvectors_to_rays(mu)

In [12]:
try:
    plot.close()
except:
    pass
plot = k3d.plot(camera_auto_fit=True, antialias=True)

for i, el in enumerate(sys.elements):
    plot += el.plot(opacity=0.4)

for i, el in enumerate(sys.elements):
    plot_element_ax(el, plot, length=5.)

plot_rays(ray, plot)
plot.display()

C:\ProgramData\Anaconda3\lib\site-packages\traittypes\traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

In [9]:
rays_interaction_factory(LensCavQuartic, 'lens_dist')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.1, description='ar', layout=Layout(height='30px', width='80%'), max=…

In [10]:
from scipy.optimize import minimize_scalar, brentq

In [69]:
#coefficients_interaction_factory(LensCavQuartic, coefpars={'c4': 3.63e-3, 'c6': 1e-4})
coefficients_interaction_factory(LensCavQuartic, coefpars={'c4': res.x[0], 'c6': res.x[1], 'lens_dist': Ldeg}, negative=False)

c:\users\lukas\box\lukas\03_theory\abcd\sloppy\src\sloppy\tools.py:264: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

{'c4': -3, 'c6': -4, 'lens_dist': 1}


interactive(children=(FloatSlider(value=3.5502199345569028, description='c4', layout=Layout(height='30px', wid…

In [31]:
from matplotlib.colors import LogNorm

In [52]:
cavfct = LensCavQuartic
degmodenum = 1
Nrt = 30
def get_dev(r, **kwargs):
    elements = cavfct(**kwargs)
    sys = RaySystem(elements)
    system = sys.abcd
    mu1, mu2 = system.q
    waist = system.waist_at(0)[0]
    mu = mu1 if degmodenum==1 else mu2
    rmu = np.linalg.norm(np.real(mu[:2]))
    mu = np.real(r*waist/rmu*mu)
    ray0 = sys.screen.eigenvectors_to_rays(mu)
    traj_hit = sys.propagate(ray0, Nrt=Nrt, at_screen=True)
    hit_scr = sys.screen.r_to_screen_coords(traj_hit[:,0,0,:])
    hit_1 = hit_scr[::3,:]
    hit_2 = hit_scr[1::3,:]
    hit_3 = hit_scr[2::3,:]
    #approximate path length described by dots through piecwise linear distance
    dev = np.linalg.norm(hit_1[1:,:]-hit_1[:-1,:], axis=1).sum() +\
            np.linalg.norm(hit_2[1:,:]-hit_2[:-1,:], axis=1).sum() +\
            np.linalg.norm(hit_3[1:,:]-hit_3[:-1,:], axis=1).sum()
    return dev


res = degeneracy_length(cavfct, 'lens_dist', 1e-1)
Ldeg = res.x
print(res.x)

#cavargs = {'lens_dist': Ldeg, 'c6': 0.}
#param = 'c4'
#rng = (3.4e-3, 3.8e-3)
cavargs = {'lens_dist': Ldeg, 'c4': 3.575e-3}
param = 'c6'
rng = (1e-4, 2.5e-4)

rs = np.linspace(np.finfo(np.float32).eps,12., 100)
ps = np.linspace(*rng, 100)
devs = np.zeros((len(rs), len(ps)))
for i, r in enumerate(rs):
    for j, p in enumerate(ps):
        args = cavargs.copy()
        args[param] = p
        try:
            d = get_dev(r=r, **args)
        except:
            devs[i,j] = np.nan
        else:
            devs[i,j] = d

25.12982681286276


In [54]:
plt.figure()
plt.pcolormesh(rs, ps, devs.T, norm=LogNorm(vmin=devs.min()*1e10, vmax=devs.max()))
plt.colorbar()
plt.show()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [57]:
cavfct = LensCavQuartic
degmodenum = 1
Nrt = 30


res = degeneracy_length(cavfct, 'lens_dist', 1e-1)
Ldeg = res.x
print(res.x)

cavargs = {'lens_dist': Ldeg}
parama = 'c4'
rnga = (3.4e-3, 3.8e-3)
paramb = 'c6'
rngb = (1e-4, 2.5e-4)

rs = np.linspace(*rnga, 100)
ps = np.linspace(*rngb, 100)
devs = np.zeros((len(rs), len(ps)))
for i, r in enumerate(rs):
    for j, p in enumerate(ps):
        args = cavargs.copy()
        args[parama] = r
        args[paramb] = p
        try:
            d = get_dev(r=10., **args)
        except:
            devs[i,j] = np.nan
        else:
            devs[i,j] = d

25.12982681286276


In [58]:
plt.figure()
plt.pcolormesh(rs, ps, devs.T, norm=LogNorm(vmin=devs.min(), vmax=devs.max()))
plt.colorbar()
plt.show()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [63]:
def deviation_tot(x):
    rs = np.linspace(np.finfo(np.float32).eps, 10., 50)
    Ls = np.zeros_like(rs)
    for i, r in enumerate(rs):
        try:
            d = get_dev(r=r, c4=x[0], c6=x[1], lens_dist=Ldeg)
        except:
            Ls[i] = np.nan
        else:
            Ls[i] = d
    return np.abs(np.sum(Ls))

In [70]:
bounds = [(2.2e-3, 4e-3), (0., 3e-4)]
res = minimize(deviation_tot, x0=[3.5e-3, 1.5e-4], bounds=bounds)
res

      fun: 1.151691970975214
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([0.00098432, 0.0014144 ])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 36
      nit: 8
   status: 0
  success: True
        x: array([0.00355023, 0.00013696])

In [65]:
def plot_cs(rmax=12., Npts=100, **kwargs):
    rs = np.linspace(np.finfo(np.float32).eps, rmax, Npts)
    Ls = np.zeros_like(rs)
    
    for i, r in enumerate(rs):
        try:
            d = get_dev(r=r, **kwargs)
        except:
            Ls[i] = np.nan
        else:
            Ls[i] = d
    fig, ax = plt.subplots()
    line = ax.plot(rs, Ls)[0]
    plt.title('Deviation vs radius')
    plt.xlabel(r'$r_{in}$ [waists]')
    plt.ylabel(r'$dev$ [mm]')
    plt.show()

In [71]:
plot_cs(c4=res.x[0], c6=res.x[1], lens_dist=Ldeg)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  if sys.path[0] == '':


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
def degeneracy_par_ray(cavfct, parname, r=0, scanrange=1e-3, p0=None):
    def get_dev(p, r):
        elements = cavfct(**{parname: p})
        sys = RaySystem(elements)
        system = sys.abcd
        mu1, mu2 = system.q
        waist = waist0 #system.waist_at(0)[0] FIX
        rmu = np.linalg.norm(np.real(mu2[:2]))
        mu = np.real(r*waist/rmu*mu2)#np.real(r*mu2)
        ray0 = sys.screen.eigenvectors_to_rays(mu)
        traj_hit = sys.propagate(ray0, Nrt=30, at_screen=True)
        hit_scr = sys.screen.r_to_screen_coords(traj_hit[:,0,0,:])
        hit_1 = hit_scr[::3,:]
        return np.var(hit_1, axis=0).sum()
    if p0 is None:
        p0 = inspect.signature(cavfct).parameters[parname].default
    if not isinstance(scanrange, tuple):
        scanrange = ((1-scanrange)*p0, (1+scanrange)*p0)
    res = minimize_scalar(get_dev, bounds=scanrange, args=(r), method='bounded')
    return res

In [22]:
rs = np.linspace(np.finfo(np.float32).resolution,8., 100)
Ps = np.zeros_like(rs)
pcur = 0.
for i, r in enumerate(rs):
    try:
        p = degeneracy_par_ray(cavfct, 'c4', r=r, scanrange=(0., 1e-2), p0=pcur).x
    except:
        Ps[i] = np.nan
    else:
        Ps[i] = p
        #pcur = p

In [23]:
plt.figure()
plt.plot(rs, Ps)
plt.title('Optimal C4 coefficient vs radius')
plt.xlabel(r'$r_{in}$ [waists]')
plt.ylabel(r'$L_{deg}$ [mm]')
#plt.axhline(Ldeg, color='grey')
#plt.plot(rs, np.polyval(coef, rs), '--')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
def plot_config(l=0., ar=0.0, br=0.0, ap=0., bp=0., Nrt=500, ms = 8., **kwargs):
        fig, (ax, bx, cx) = plt.subplots(ncols=3, figsize=(11,3.3), sharex=True, sharey=True)
        linea = ax.scatter([], [], c=[], cmap='jet', s=ms)
        linec = cx.scatter([], [], c=[], cmap='jet', s=ms)
        lineb = bx.scatter([], [], c=[], cmap='jet', s=ms)
        ax.set_title('Raytracing')
        bx.set_title('ABCD')
        cx.set_title('ABCD (Extracted)')
        plt.xlabel('x [mm]')
        plt.ylabel('y [mm]')
        kwargs.update({parname: l})
        elements = cavfct(**kwargs)
        sys = RaySystem(elements)
        system = sys.abcd
        mu1, mu2 = system.q
        waist = system.waist_at(0)[0]
        rmu1 = np.linalg.norm(np.real(mu1[:2]))
        rmu2 = np.linalg.norm(np.real(mu2[:2]))
        mu = np.real(ar*waist/rmu1*mu1*np.exp(1j*ap) + br*waist/rmu2*mu2*np.exp(1j*bp))
        ray0 = sys.screen.eigenvectors_to_rays(mu)
 
        traj_hit = sys.propagate(ray0, Nrt=Nrt, at_screen=True)
        hit_scr = sys.screen.r_to_screen_coords(traj_hit[:,0,0,:])

        linea.set_offsets(hit_scr[:,:])
        linea.set_array(makeidx(hit_scr))
        
        hit_m = propagate_ABCD(mu, system.abcd_rt, Nrt=Nrt)
        lineb.set_offsets(hit_m[:,0:2])
        lineb.set_array(makeidx(hit_m))
        
        #abcd_fd = sys.extract_ABCD(epsr=1e-4, epss=1e-4, Nrt=1) 
        #hit_fd = propagate_ABCD(mu, abcd_fd, Nrt=Nrt)
        #linec.set_offsets(hit_fd[:,0:2])
        #linec.set_array(makeidx(hit_fd))
        
        linea.set_clim(vmin=0, vmax=Nrt)
        lineb.set_clim(vmin=0, vmax=Nrt)
        linec.set_clim(vmin=0, vmax=Nrt)
            
        ax.set_xlim(1.5*np.nanmin(hit_scr[:,0]), 1.5*np.nanmax(hit_scr[:,0]))
        ax.set_ylim(1.5*np.nanmin(hit_scr[:,1]), 1.5*np.nanmax(hit_scr[:,1]))
        return fig

In [26]:
rw = 5.
interp=True
if interp:
    lw = np.polyval(coef, rw)
else:
    idx = np.where(rs>rw)[0][0]
    lw = Ls[idx]
    rw = rs[idx]
f = plot_config(l=lw, br=rw, Nrt=500, ms=4, c4=0.416e-2)
f.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [117]:
plt.close("all")

## Eigenray

In [65]:
def plot_config_ray(mu, l=0., Nrt=500, ms = 8., **kwargs):
        fig, (ax, bx, cx) = plt.subplots(ncols=3, figsize=(11,3.3), sharex=True, sharey=True)
        linea = ax.scatter([], [], c=[], cmap='jet', s=ms)
        linec = cx.scatter([], [], c=[], cmap='jet', s=ms)
        lineb = bx.scatter([], [], c=[], cmap='jet', s=ms)
        ax.set_title('Raytracing')
        bx.set_title('ABCD')
        cx.set_title('ABCD (Extracted)')
        plt.xlabel('x [mm]')
        plt.ylabel('y [mm]')
        elements = cavfct(**{parname: l})
        sys = RaySystem(elements)
        ray0 = sys.screen.eigenvectors_to_rays(mu)

        traj_hit = sys.propagate(ray0, Nrt=Nrt, at_screen=True)
        hit_scr = sys.screen.r_to_screen_coords(traj_hit[:,0,0,:])

        linea.set_offsets(hit_scr[:,:])
        linea.set_array(makeidx(hit_scr))
        
        hit_m = propagate_ABCD(mu, system.abcd_rt, Nrt=Nrt)
        lineb.set_offsets(hit_m[:,0:2])
        lineb.set_array(makeidx(hit_m))
        
        abcd_fd = sys.extract_ABCD(epsr=1e-4, epss=1e-4, Nrt=1) 
        hit_fd = propagate_ABCD(mu, abcd_fd, Nrt=Nrt)
        linec.set_offsets(hit_fd[:,0:2])
        linec.set_array(makeidx(hit_fd))
        
        linea.set_clim(vmin=0, vmax=Nrt)
        lineb.set_clim(vmin=0, vmax=Nrt)
        linec.set_clim(vmin=0, vmax=Nrt)
            
        ax.set_xlim(1.5*np.nanmin(hit_scr[:,0]), 1.5*np.nanmax(hit_scr[:,0]))
        ax.set_ylim(1.5*np.nanmin(hit_scr[:,1]), 1.5*np.nanmax(hit_scr[:,1]))
        return fig

In [74]:
#mu = np.array([5e-3, 0., 0, 0])
system = sys.abcd
mu1, mu2 = system.q
waist = system.waist_at(0)[0]
rmu = np.linalg.norm(np.real(mu2[:2]))
r = 5e-3
mu = np.real(r*waist/rmu*mu2)
ray0 = sys.screen.eigenvectors_to_rays(mu)
lr = np.zeros((2, 1, 3))
#lr[1,:,:] = 0.05
lr = 0.05
#reig = sys.find_eigenray(ray0, lr=lr, Nrt=30, debug=True)
reig = sys.find_eigenray_mpe(ray0, lr=lr, Nrt=30, debug=True)
mueig = sys.screen.rays_to_eigenvectors(reig)
mueig

Finished in 49 steps, reached tol 5.473e-05
Finished in 49 steps, reached tol 2.166e-09
Finished in 49 steps, reached tol 4.613e-08
Finished in 49 steps, reached tol 3.327e-08
Finished in 49 steps, reached tol 3.617e-08


array([[ 6.07047033e-05,  7.65926464e-05, -1.10603097e-04,
        -3.29846840e-04]])

In [75]:
f = plot_config_ray(np.squeeze(mueig), Ldeg)
f.show()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [81]:
def degeneracy_length_eigenray(cavfct, parname, r=0, scanrange=1e-3, La=None):
    def get_dev(l, r):
        elements = cavfct(**{parname: l})
        sys = RaySystem(elements)
        system = sys.abcd
        mu1, mu2 = system.q
        waist = system.waist_at(0)[0]
        rmu = np.linalg.norm(np.real(mu2[:2]))
        mu = np.real(r*waist/rmu*mu2)
        ray0 = sys.screen.eigenvectors_to_rays(mu)
        lr = 0.05
        reig = sys.find_eigenray(ray0, lr=lr, Nrt=3, debug=False)
        #reig = sys.find_eigenray_mpe(ray0, lr=lr, Nrt=30, debug=False)
        mueig = sys.screen.rays_to_eigenvectors(reig)
        
        ray1 = sys.screen.eigenvectors_to_rays(mu)
        traj_hit = sys.propagate(ray1, Nrt=30, at_screen=True)
        hit_scr = sys.screen.r_to_screen_coords(traj_hit[:,0,0,:])
        hit_1 = hit_scr[::3,:]
        return np.var(hit_1, axis=0).sum()
    if La is None:
        La = inspect.signature(cavfct).parameters[parname].default
    res = minimize_scalar(get_dev, bounds=((1-scanrange)*La, (1+scanrange)*La), args=(r,), method='bounded')
    return res

In [82]:
degeneracy_length_eigenray(cavfct, parname, r=1e-3, scanrange=1e-3, La=Ldeg)

     fun: 2.020474060964267e-08
 message: 'Solution found.'
    nfev: 10
  status: 0
 success: True
       x: 21.506287396753475

In [83]:
rs = np.linspace(0,8., 100)
Ls = np.zeros_like(rs)
lcur = Ldeg
for i, r in enumerate(rs):
    try:
        l = degeneracy_length_eigenray(cavfct, parname, r=r, scanrange=1e-2, La=lcur).x
    except:
        Ls[i] = np.nan
    else:
        Ls[i] = l
        lcur = l

In [84]:
idx = np.where(rs>5.)[0][0] #only for up to
coef = np.polyfit(rs[:idx], Ls[:idx], deg=4)
coef

array([-2.43816029e-04,  2.60880858e-03, -2.34249747e-02,  1.36223586e-02,
        2.15193646e+01])

In [85]:
plt.figure()
plt.plot(rs, Ls)
plt.title('Degeneracy length vs radius')
plt.xlabel(r'$r_{in}$ [waists]')
plt.ylabel(r'$L_{deg}$ [mm]')
plt.axhline(Ldeg, color='grey')
plt.plot(rs, np.polyval(coef, rs), '--')
plt.show()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [87]:
rw = 5.5
interp=False
if interp:
    lw = np.polyval(coef, rw)
else:
    idx = np.where(rs>rw)[0][0]
    lw = Ls[idx]
    rw = rs[idx]
f = plot_config(l=lw, br=rw, Nrt=500, ms=4)
f.show()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …